Cálculos utilizado no desenvolvimento do projeto de formatura do departamento de Engenharia de Sistemas Eletrônicos pela Escola Politécnica da Universidade de São Paulo

Autor: Marcus Vinicius Gonçalves Mendes
N° USP: 9349516
Orientadores: Marcelo Knörich Zuffo e Roseli de Deus Lopes
Orientador externo: Alberto Ferreira de Sousa

In [7]:
import math
from matplotlib import pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [64]:
# Constants
# Values: 0, 2.5, 6, 11
adc_attenuation = 11
vcc = 5

r_values = [27e3, 33e3, 47e3, 68e3, 82e3, 100e3, 150e3, 220e3, 330e3,470e3, 560e3, 680e3, 1e6, 5.6e6]

In [82]:
# ESP32 data

# ESP32 ADC measurable input voltage ranges, classified by attenuation levels, in Volts
# Source: https://docs.espressif.com/projects/esp-idf/en/latest/esp32/api-reference/peripherals/adc.html

esp32_adc = {  0: {"v_min": 0.10, "v_max": 0.95, "average": (0.10 + 0.95) / 2, "excursion": (0.95 - 0.10)},
             2.5: {"v_min": 0.10, "v_max": 1.25, "average": (0.10 + 1.25) / 2, "excursion": (1.25 - 0.10)},
               6: {"v_min": 0.15, "v_max": 1.75, "average": (0.15 + 1.75) / 2, "excursion": (1.75 - 0.15)},
              11: {"v_min": 0.15, "v_max": 2.45, "average": (0.15 + 2.45) / 2, "excursion": (2.45 - 0.15)}}

In [119]:
#############################
#                           #
# Current sensor SCT013-100 #
#                           #
#############################

# Calculation to connect the SCT013 sensor with the ESP32 ADC
# Source: https://learn.openenergymonitor.org/electricity-monitoring/ct-sensors/interface-with-arduino

sct013_primary_current_max_rms = 100
sct013_number_turns = 2000


sct013_primary_current_peak = math.sqrt(2) * sct013_primary_current_max_rms
sct013_secondary_current_peak = sct013_primary_current_peak / sct013_number_turns


# Calculates the burden resistor value to close the circuit between the output and ground of SCT013
sct013_burden_resistor = ((esp32_adc[adc_attenuation]["excursion"]) / 2) / sct013_secondary_current_peak
print("SCT013 Burden Resitor = {:.2f} Ω".format(sct013_burden_resistor))


# Calculates the relation between resistor value to add a DC bias to be connected to SCT013 ground, in which the
# sequence is vcc -> r1 -> r2 -> gnd
# and r1 is given in multiples of r2, and r2 can be chosen by developer
sct013_dc_bias_r1 = (vcc - esp32_adc[adc_attenuation]["average"]) / esp32_adc[adc_attenuation]["average"]
sct013_dc_bias_r2 = 1
print("R1 = {:.5f} * R2".format(sct013_dc_bias_r1))


# Calculate best combination of real values resistor
sct013_dc_bias_r1_real_index = 0
sct013_dc_bias_r2_real_index = 0
sct013_dc_bias_r_best_match = 0

for sct013_r2_index in range(len(r_values)):
    for sct013_r1_index in range(sct013_r2_index + 1, len(r_values)):
        if abs(((r_values[sct013_r1_index] / r_values[sct013_r2_index]) / sct013_dc_bias_r1) - 1) < abs(sct013_dc_bias_r_best_match - 1):
            sct013_dc_bias_r_best_match = (r_values[sct013_r1_index] / r_values[sct013_r2_index]) / sct013_dc_bias_r1
            sct013_dc_bias_r2_real_index = sct013_r2_index
            sct013_dc_bias_r1_real_index = sct013_r1_index
            
print("R1 = {:,} Ω, R2 = {:,} Ω, Percentage to ideal = {:.5} %".format(r_values[sct013_dc_bias_r1_real_index], r_values[sct013_dc_bias_r2_real_index], 100 * sct013_dc_bias_r_best_match))
sct013_v_dc_bias = vcc * r_values[sct013_dc_bias_r2_real_index] / (r_values[sct013_dc_bias_r1_real_index] + r_values[sct013_dc_bias_r2_real_index])
print("V_dc_bias = {:.2f} V".format(sct013_v_dc_bias))
print("V_signal_min = {:.4f} V, V_signal_max = {:.4f} V".format(sct013_v_dc_bias - (sct013_secondary_current_peak * sct013_burden_resistor), sct013_v_dc_bias + (sct013_secondary_current_peak * sct013_burden_resistor)))

SCT013 Burden Resitor = 16.26 Ω
R1 = 2.84615 * R2
R1 = 220,000.0 Ω, R2 = 82,000.0 Ω, Percentage to ideal = 94.265 %
V_dc_bias = 1.36 V
V_signal_min = 0.2076 V, V_signal_max = 2.5076 V


In [ ]:
i2 = i1 / n_turns
vb = i2 * rb
vadc = vb + vbias

vb = vadc - vbias
i2 = vb / rb
i2 = (vadc - vbias) / rb
i1 = i2 * nturns
i1 = ((vadc - vbias) / rb) * nturns

In [118]:
###########################
#                         #
# Voltage sensor ZMPT101B #
#                         #
###########################

# Needs fixing ################################################################################
# Needs fixing ################################################################################
# Needs fixing ################################################################################
# Verify maximum output voltage

# Calculation that should be done to connect the ZMPT101B sensor with the ESP32 ADC, if module didn't have it ebedded
# Module already has all resistors, and sampling resistor is trimpot
# Source https://innovatorsguru.com/zmpt101b/


zmpt101b_rated_input_voltage_rms = 250
zmpt101b_rated_input_current_rms = 0.00125


zmpt101b_max_output_voltage = esp32_adc[adc_attenuation]["excursion"] / (2 * math.sqrt(2))
print("Vout_max =", zmpt101b_max_output_voltage, "V")


zmpt101b_current_limiting_resistor = zmpt101b_rated_input_voltage_rms / zmpt101b_rated_input_current_rms
print("R1 (Current Limiting Transistor) = {:.2f} Ω".format(zmpt101b_current_limiting_resistor))


zmpt101b_sampling_resistor = (zmpt101b_max_output_voltage / zmpt101b_rated_input_voltage_rms) * zmpt101b_current_limiting_resistor
print("R2 (Sampling Resistor) = {:.2f} Ω".format(zmpt101b_sampling_resistor))

Vout_max = 0.8131727983645297 V
R1 (Current Limiting Transistor) = 200000.00 Ω
R2 (Sampling Resistor) = 650.54 Ω


In [107]:
###################
#                 #
# Voltage divider #
#                 #
###################

# Voltage divider with 3 resistors, in which:
# r1 + r2 + r3 = Vcc
# r2 + r3 = max adc input
# r3 = min adc input


voltage_divider_r = [(vcc - esp32_adc[adc_attenuation]["v_max"]) / esp32_adc[adc_attenuation]["v_min"], 
                     (esp32_adc[adc_attenuation]["excursion"]) / esp32_adc[adc_attenuation]["v_min"],
                     1]
print("R1, R2, R3 = {:.2f}*R3, {:.2f}*R3, {:.2f}*R3".format(voltage_divider_r[0], voltage_divider_r[1], voltage_divider_r[2]))

R1, R2, R3 = 17.00*R3, 15.33*R3, 1.00*R3


In [121]:
# To calculate a voltage divider output anf offset given input and resistance values
# Source: https://electronics.stackexchange.com/questions/166626/how-to-add-a-dc-bias

def parallel(r1, r2):
    return r1 * r2 / (r1 + r2)

def voltage_divider (v_in, r1, r2):
    return v_in * r2 / (r1 + r2)

def dc_bias_voltage_divider(v_signal, vcc, r1, r2, r3):
    v_out = voltage_divider(v_signal, r1, parallel(r2, r3))
    v_bias = voltage_divider(vcc, r3, parallel(r1, r2))
    
    return (v_out, v_bias)

voltage_divider_out_bias = dc_bias_voltage_divider(2.5, 5, 33e3, 29765, 506000)
print("dc_bias = {:.2f} V\nVout_max = {:.2f} V".format(voltage_divider_out_bias[1], voltage_divider_out_bias[1] + (2 * voltage_divider_out_bias[0])))

dc_bias = 0.15 V
Vout_max = 2.45 V


In [116]:
# To calculate resistance values of a voltage divider with offset given input and desired output and offset
# Derived from
#       Source: https://electronics.stackexchange.com/questions/166626/how-to-add-a-dc-bias


r1 = 33e3
vs = vcc/2
vo = esp32_adc[adc_attenuation]["excursion"] / 2
vbias = esp32_adc[adc_attenuation]["v_min"]


r2 = r1 * vo * vcc / ((vs * (vcc - vbias)) - (vo * vcc))
r3 = r1 / ((vs / vo) - 1 - (r1/r2))
print("r1 = {:,} Ω\nr2 = {:,} Ω\nr3 = {:,} Ω".format(round(r1), round(r2), round(r3)))

r1 = 33,000 Ω
r2 = 29,765 Ω
r3 = 506,000 Ω


In [3]:
def voltage_divider(V, R1, R2):
    return (V * R2 / (R1 + R2))

In [5]:
print(voltage_divider(3.3, 22e3, 10e3))
print(voltage_divider(4.65, 10e3, 10e3))

1.03125
2.325
